In [145]:
import json
import os
import re
from shutil import copyfile

from bids.layout.writing import build_path

In [111]:
deriv_path = os.getcwd()
out = deriv_path.replace(os.path.basename(deriv_path), 'new_fmriprep')
os.makedirs(out, exist_ok=True)

In [142]:
# templates for matching the old fmriprep files
BIDS_NAME = (
    r'^(.*\/)?'
    r'sub-(?P<subject>[a-zA-Z0-9]+)'
    r'(_ses-(?P<session>[a-zA-Z0-9]+))?'
    r'(_task-(?P<task>[a-zA-Z0-9]+))?'
    r'(_acq-(?P<acq>[a-zA-Z0-9]+))?'
    r'(_rec-(?P<rec>[a-zA-Z0-9]+))?'
    r'(_run-(?P<run>[a-zA-Z0-9]+))?')

DERIV_NAME = (
    r'_(?P<suffix>[a-zA-Z0-9]+)'
    r'(_space-(?P<space>[a-zA-Z0-9]+))?'
    r'(_class-(?P<class>[a-zA-Z0-9]+))?'
    r'(_target-(?P<target>[a-zA-Z0-9]+))?'
    r'(_variant-(?P<variant>[a-zA-Z0-9]+))?'
    r'_(?P<desc>[a-zA-Z0-9]+)'
    r'\.(?P<ext>(?!svg).*)')

DERIV_REGEX = re.compile(r''.join([BIDS_NAME, DERIV_NAME]))

# template to build the "correct" derivative file
PATH_PATTERN = (
    'sub-{subject}'
    '[_ses-{session}]'
    '[_task-{task}]'
    '[_acq-{acq}]'
    '[_rec-{rec}]'
    '[_run-{run}]'
    '[_from-{fspace}]'
    '[_to-{tspace}]'
    '[_mode-{mode}]'
    '[_hemi-{hemi}]'
    '[_space-{space}]'
    '[_label-{label}]'
    '[_desc-{desc}]'
    '_{suffix}'
    '.{ext}')

In [143]:
rename_files = {}
for root, _, files in os.walk(deriv_path):
    for file in files:
        match = re.search(deriv_pattern, file)
        if match is not None:
            file_dict = match.groupdict()
            # change brainmask to desc-brain_mask
            if file_dict.get('desc') == 'brainmask':
                file_dict['desc'] = 'brain'
                file_dict['suffix'] = 'mask'
            # variant is now desc
            if file_dict.get('variant'):
                file_dict['desc'] = file_dict.pop('variant')
            # different formats of transformation files
            if file_dict.get('space') and file_dict.get('target'):
                file_dict['fspace'] = file_dict.pop('space')
                file_dict['tspace'] = file_dict.pop('target')
                file_dict['mode'] = 'image'
                file_dict['suffix'] = 'xfm'
                del file_dict['desc']
            if file_dict.get('target'):
                file_dict['fspace'] = file_dict.pop('suffix')
                file_dict['tspace'] = file_dict.pop('target')
                file_dict['mode'] = 'image'
                file_dict['suffix'] = 'xfm'
                del file_dict['desc']
            # segmentations
            if file_dict.get('class'):
                file_dict['label'] = file_dict.pop('class')
                file_dict['suffix'] = 'probseg'
                del file_dict['desc']
            if file_dict.get('desc') == 'dtissue':
                file_dict['suffix'] = 'dseg'
                del file_dict['desc']
            # freesurfer hemisphere files
            if file_dict['ext'].startswith('L') or file_dict['ext'].startswith('R'):
                file_dict['hemi'] = file_dict['ext'][0]
                file_dict['ext'] = file_dict['ext'][2:]
                file_dict['suffix'] = file_dict.pop('desc')
            # aroma files
            if file_dict.get('desc') == 'MELODICmix':
                file_dict['desc'] = 'MELODIC'
                file_dict['suffix'] = 'mixing'
            if file_dict.get('desc') == 'AROMAnoiseICs':
                file_dict['suffix'] = file_dict['desc']
                del file_dict['desc']
            # confounds file change
            if file_dict.get('desc') == 'confounds':
                file_dict['suffix'] = 'regressors'
           
            # write the file to the new directory
            new_file = build_path(file_dict, path_pattern)
            new_root = root.replace(deriv_path, out)
            new_path = os.path.join(new_root, new_file)
            old_path = os.path.join(root, file)
            rename_files[old_path] = new_path
            os.makedirs(new_root, exist_ok=True)
            copyfile(old_path, new_path)

with open('data_transfer.json', 'w') as fp:
    json.dump(rename_files, fp)

In [144]:
rename_files

{'/media/Data/Documents/devel/fmriprepTransfer/sub-01/anat/sub-controlGE140_T1w_preproc.nii.gz': '/media/Data/Documents/devel/new_fmriprep/sub-01/anat/sub-controlGE140_desc-preproc_T1w.nii.gz',
 '/media/Data/Documents/devel/fmriprepTransfer/sub-01/anat/sub-controlGE140_T1w_brainmask.nii.gz': '/media/Data/Documents/devel/new_fmriprep/sub-01/anat/sub-controlGE140_desc-brain_mask.nii.gz',
 '/media/Data/Documents/devel/fmriprepTransfer/sub-01/anat/sub-controlGE140_T1w_class-CSF_probtissue.nii.gz': '/media/Data/Documents/devel/new_fmriprep/sub-01/anat/sub-controlGE140_label-CSF_probseg.nii.gz',
 '/media/Data/Documents/devel/fmriprepTransfer/sub-01/anat/sub-controlGE140_T1w_class-GM_probtissue.nii.gz': '/media/Data/Documents/devel/new_fmriprep/sub-01/anat/sub-controlGE140_label-GM_probseg.nii.gz',
 '/media/Data/Documents/devel/fmriprepTransfer/sub-01/anat/sub-controlGE140_T1w_class-WM_probtissue.nii.gz': '/media/Data/Documents/devel/new_fmriprep/sub-01/anat/sub-controlGE140_label-WM_probseg.

In [117]:
for old_file, new_file in rename_files.items():
    os.makedirs(os.path.dirname(new_file), exist_ok=True)
    copyfile(old_file, new_file)


In [121]:
from bids import BIDSLayout


In [139]:
layout = BIDSLayout(os.path.join(deriv_path, 'new_fmriprep'), config=['derivatives', 'bids'])

In [140]:
layout.files

{'/media/Data/Documents/devel/fmriprepTransfer/new_fmriprep/dataset_description.json': <BIDSFile filename='dataset_description.json'>,
 '/media/Data/Documents/devel/fmriprepTransfer/new_fmriprep/sub-controlGE140/anat/sub-controlGE140_hemi-R_pial.surf.gii': <BIDSFile filename='sub-controlGE140/anat/sub-controlGE140_hemi-R_pial.surf.gii'>,
 '/media/Data/Documents/devel/fmriprepTransfer/new_fmriprep/sub-controlGE140/anat/sub-controlGE140_desc-affine_T1w.txt': <BIDSFile filename='sub-controlGE140/anat/sub-controlGE140_desc-affine_T1w.txt'>,
 '/media/Data/Documents/devel/fmriprepTransfer/new_fmriprep/sub-controlGE140/anat/sub-controlGE140_desc-brain_mask.nii.gz': <BIDSFile filename='sub-controlGE140/anat/sub-controlGE140_desc-brain_mask.nii.gz'>,
 '/media/Data/Documents/devel/fmriprepTransfer/new_fmriprep/sub-controlGE140/anat/sub-controlGE140_desc-preproc_T1w.nii.gz': <BIDSFile filename='sub-controlGE140/anat/sub-controlGE140_desc-preproc_T1w.nii.gz'>,
 '/media/Data/Documents/devel/fmripre